In [1]:
from openai import OpenAI
import yaml
import json

In [3]:
with open('config.yaml', 'r') as yml:
    config = yaml.safe_load(yml)

In [6]:
client = OpenAI(
    api_key = config["oai"]["key"], # 取得したAPIキー
    # base_url= <URL> # Azure OpenAI Serviceを使う場合は必要
)

In [4]:
tools = [
    {
        "type": "function",
        "name": "web_search",
        "description": "指定された複数のクエリでWeb検索を行い、各結果を要約して返します。",        
        "parameters": {
            "type": "object",
            "properties": {
                "queries": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "検索するクエリ文字列のリスト"
                }
            },
            "required": ["queries"]
        }
    }
]

In [12]:
tool_def = [
  { 
    "type": "function",
    "name": "generate_rag_search_components",
    "description": "ユーザ入力に基づき、RAG（検索拡張生成）のためのステップバック、関連知識、および検索クエリを生成する。",
    "parameters": {
      "type": "object",
      "properties": {
        "step_back": {
          "type": "string",
          "description": "ユーザ要求を一段抽象化し、意図を明確にする簡潔な再定式化です。"
        },
        "background_knowledge": {
          "type": "string",
          "description": "より効果的な検索クエリを作成するために役立つドメイン知識または常識です。"
        },
        "queries": {
          "type": "array",
          "items": {
            "type": "string"
          },
          "description": "ユーザ要求に関連する情報を取得するための検索エンジン用クエリ文のリストです。キーワードではなく日本語の文章のリストで生成します。必要に応じて複数のクエリを生成してください。"
          
        }
      },
      "required": ["step_back", "background_knowledge", "queries"]
    }
  }
]

question = "AzureでGPTを使う方法とAWSでLLMを使う方法の比較をしたい。"

messages=[
        {"role": "system", "content": "適宜ツールを使ってユーザに回答してください。"},
        {"role": "user", "content": question}
    ]
response = client.responses.create(
    model="gpt-5-mini",
    input=messages,
    tools=tool_def,
    tool_choice={"type":"function","name":"generate_rag_search_components"}, # 実験のため呼び出すツールを強制
)

arguments = response.output[1].arguments

print("argument: ")
print(json.dumps(json.loads(arguments), indent=2, ensure_ascii=False))

argument: 
{
  "step_back": "Azure上でOpenAI系（GPT）を使う方法と、AWS上でLLM（Bedrock/SageMaker/セルフホスティング）を使う方法を機能・運用・コスト・セキュリティなどの観点で比較したいという意図。",
  "background_knowledge": "比較のために押さえるべき背景知識：\n- AzureはOpenAIと提携しAzure OpenAI Service（Azure AI Studio含む）でGPT系モデルをマネージドで提供。モデルバージョン、利用制限、リージョンごとの可用性、VNet/Private Link、Enterprise契約やデータ取り扱い協定が重要。\n- AWSはBedrockで複数のファウンデーションモデル（Anthropic, Cohere, AI21, Amazon Titan等）をマネージド提供、SageMakerでは任意モデルのトレーニング/デプロイが可能。セルフホストやコンテナ化（ECR/ECS/EKS/EC2 GPU）での柔軟性が高い。\n- カスタマイズ手段：マネージドの微調整（Fine-tuning）、プロンプトデザイン、ファインチューニング代替（RL, LoRA）、プライベートエンドポイントやオンプレ/エッジでのデプロイが比較軸。\n- 検索・RAG連携：Azure Cognitive SearchやAmazon Kendra/OpenSearch + k-NN、外部ベクタDB（Pinecone, Milvus, Weaviate）との統合とコスト。\n- 運用面：スケーリング（オートスケーリング）、デプロイ費用（インスタンス料金 vs リクエスト課金）、監視・ログ（Azure Monitor, CloudWatch）、モデルガバナンス、監査証跡。\n- セキュリティ・コンプライアンス：VNet/PrivateLink、KMSによる暗号化、SOC/ISO/HIPAA等認証対応、データ保持と使用に関するプロバイダーポリシー。\n- 価格体系：トークン課金（OpenAI系）やリクエスト/時間課金（Bedrock/SageMaker）、インフラ（GPUインスタンス）コストの違い。\n- レイテンシとリージョン：リージョンとエッジ要件が遅延に与える影響。\n\

In [9]:
response.output

[ResponseReasoningItem(id='rs_00b2ea661abbf3a6006946e01681108194bf51c6393b4840b5', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseOutputMessage(id='msg_00b2ea661abbf3a6006946e02546488194ba19f27821b563fd', content=[ResponseOutputText(annotations=[], text='いい比較ですね。以下は「AzureでGPTを使う（主に Azure OpenAI Service）」と「AWSでLLMを使う（主に Amazon Bedrock / SageMaker を想定）」の主要な違い・特徴を整理したものです。導入判断に使えるチェックリストと典型的なアーキテクチャ例も付けます。\n\n要約（結論）\n- すぐにOpenAI系（GPT）モデルを使い、Microsoftエコシステム（Azure AD、M365、Power Platform）と強く連携したい → Azure OpenAI Serviceが自然な選択。\n- 複数ベンダーのファウンデーションモデル（Anthropic/Meta/Cohere/独自モデル/AWS Titan 等）を比較利用したい、または自分で大モデルをトレーニング/ホストしたい → AWS（Bedrock + SageMaker）の方が柔軟性が高い。\n\n比較ポイント\n\n1) 提供サービスとモデルの選択肢\n- Azure\n  - 主に Azure OpenAI Service 経由でOpenAIのモデル（GPT-3.5/GPT-4系、embeddingsなど）を提供。\n  - Azure AI Studio による管理 GUI、デプロイ、テスト、プロンプト管理など。\n  - Microsoft のCognitive Servicesと統合可能。\n- AWS\n  - Amazon Bedrock：複数ベンダーのファウンデーションモデル（Anthropic, Cohere, Mistral等）＋AWSのTitanをマネ

PDFのhtml化

In [1]:
from pathlib import Path
import os

# pip install pymupdf
import fitz  # PyMuPDF

def pdf_to_html(
    input_pdf: str | Path,
    output_html: str | Path,
    *,
    standalone: bool = True
) -> None:
    """
    PDF を HTML に変換して保存する。

    Parameters
    ----------
    input_pdf : str | Path
        入力 PDF ファイル
    output_html : str | Path
        出力 HTML ファイル
    standalone : bool
        <!DOCTYPE html> 付きの完全な文書にするかどうか
    """
    input_pdf = Path(input_pdf)
    output_html = Path(output_html)

    if not input_pdf.exists():
        raise FileNotFoundError(f"入力PDFが見つかりません: {input_pdf}")

    print(f"PDF を読み込み中: {input_pdf}")
    doc = fitz.open(input_pdf)

    html_parts: list[str] = []
    for page_num, page in enumerate(doc, start=1):
        html_parts.append(page.get_text("html"))
        print(f"  - ページ {page_num} を変換")

    # HTML ドキュメントを構築
    if standalone:
        html = "<!DOCTYPE html>\n<html>\n<body>\n" + "\n".join(html_parts) + "\n</body>\n</html>"
    else:
        html = "\n".join(html_parts)

    # 出力ディレクトリを作成
    output_html.parent.mkdir(parents=True, exist_ok=True)

    # 保存
    with output_html.open("w", encoding="utf-8", newline="\n") as f:
        f.write(html)

    print(f"HTML 変換が完了しました: {output_html}")


# ------------------ 実行例 ------------------
if __name__ == "__main__":
    INPUT_PDF = Path("./reference/RAG_Sample_Doc.pdf")
    OUTPUT_HTML = Path("./result/pdf2html/RAG_Sample_Doc.html")

    pdf_to_html(INPUT_PDF, OUTPUT_HTML)

PDF を読み込み中: reference\RAG_Sample_Doc.pdf
  - ページ 1 を変換
  - ページ 2 を変換
HTML 変換が完了しました: result\pdf2html\RAG_Sample_Doc.html


Wordドキュメントのhtml化

In [3]:
from pathlib import Path
import os
import pandoc  # pip install pandoc

def docx_to_html(
    input_path: str | Path,
    output_path: str | Path | None = None,
    *,
    standalone: bool = True,
    extract_media_dir: str | Path | None = None
) -> str:
    """
    DOCX を HTML に変換する。

    Parameters
    ----------
    input_path : str | Path
        入力 .docx ファイル
    output_path : str | Path | None
        出力 HTML ファイル名。None の場合は文字列を返すだけ。
    standalone : bool
        HTML を <!DOCTYPE html> 付きの完全な文書にするかどうか。
    extract_media_dir : str | Path | None
        画像などを外部ファイルとして保存したい場合の取り出し先ディレクトリ (--extract-media)。
    Returns
    -------
    str
        生成された HTML（output_path を与えた場合でも文字列を返す）
    """
    # 1. 読み込み時のオプション（--extract-media は reader 専用）
    reader_opts: list[str] = []
    if extract_media_dir:
        abs_media_dir = os.path.abspath(extract_media_dir)
        reader_opts.append(f"--extract-media={abs_media_dir}")
        print(f"Pandoc reader オプション: --extract-media={abs_media_dir}")

    # 2. 読み込み
    doc = pandoc.read(file=input_path, format="docx", options=reader_opts)

    # 3. 書き出し時のオプション
    writer_opts: list[str] = []
    if standalone:
        writer_opts.append("--standalone")
    print(f"Pandoc writer オプション: {writer_opts}")

    # 4. 書き出し
    html = pandoc.write(
        doc,
        file=output_path,   # None なら戻り値のみ
        format="html",
        options=writer_opts
    )

    # 5. メディアファイルの確認
    if extract_media_dir:
        media_files = list(Path(extract_media_dir).rglob("*"))
        print(f"抽出されたメディアファイル数: {len(media_files)}")
        for f in media_files:
            if f.is_file():
                print(f"  - {f}")

    return html


# ------------- 使い方例 -------------
if __name__ == "__main__":
    input_file = "./reference/RAG_Sample_Doc.docx"
    if not os.path.exists(input_file):
        print(f"入力ファイルが見つかりません: {input_file}")
    else:
        print(f"入力ファイル確認: {input_file}")

    html_text = docx_to_html(
        input_file,
        output_path="result/word2html/RAG_Sample_Doc.html",
        extract_media_dir="result/word2html/media"
    )
    print("HTML 変換が完了しました。")


入力ファイル確認: ./reference/RAG_Sample_Doc.docx
Pandoc reader オプション: --extract-media=c:\Users\hirosatogamo\iCloudDrive\hiro_project\prompt_book\code\result\word2html\media
Pandoc writer オプション: ['--standalone']
抽出されたメディアファイル数: 2
  - result\word2html\media\media\image1.png
HTML 変換が完了しました。


htmlをTableのみ保持したMarkdown形式へ変換するサンプル

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from __future__ import annotations

import re
from pathlib import Path
from typing import List, Tuple

from bs4 import BeautifulSoup, Comment, NavigableString, Tag
from markdownify import markdownify as md

INPUT_HTML = Path("./result/word2html/RAG_Sample_Doc.html")
OUTPUT_MD_DIR = Path("./result/html2markdown")
OUTPUT_MD = OUTPUT_MD_DIR / "RAG_Sample_Doc.md"

TABLE_TOKEN_TEMPLATE = "§§TABLE{idx}§§"


def load_html(path: Path) -> BeautifulSoup:
    with path.open("r", encoding="utf-8") as f:
        return BeautifulSoup(f.read(), "html.parser")


def remove_css_and_related_attrs(soup: BeautifulSoup) -> None:
    # <style> と <link rel="stylesheet"> を削除
    for style_tag in soup.find_all("style"):
        style_tag.decompose()
    for link_tag in soup.find_all("link", rel=lambda v: v and "stylesheet" in v.lower()):
        link_tag.decompose()

    # コメント削除
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()

    # <input> やフォーム系タグを削除（"input" 文字が出ないように）
    for tag in soup.find_all(["input", "form", "select", "option", "textarea", "button"]):
        tag.decompose()

    # 装飾系属性を削除
    css_like_attrs = {
        "style",
        "class",
        "id",
        "width",
        "height",
        "align",
        "valign",
        "bgcolor",
        "border",
        "cellpadding",
        "cellspacing",
        "color",
        "background",
    }
    for tag in soup.find_all(True):
        for attr in list(tag.attrs):
            if attr.lower() in css_like_attrs:
                del tag.attrs[attr]


def extract_tables_and_replace_with_tokens(
    soup: BeautifulSoup,
) -> Tuple[str, List[Tuple[str, str]]]:
    tables: List[Tag] = soup.find_all("table")
    token_table_pairs: List[Tuple[str, str]] = []

    for idx, table in enumerate(tables, start=1):
        token = TABLE_TOKEN_TEMPLATE.format(idx=idx)
        table_html = str(table)
        token_table_pairs.append((token, table_html))
        table.replace_with(NavigableString(token))

    return str(soup), token_table_pairs


def html_to_markdown(html: str) -> str:
    return md(
        html,
        heading_style="ATX",
        bullets="-",
        escape_asterisks=False,
    )


def restore_tables_into_markdown(
    markdown_text: str, token_table_pairs: List[Tuple[str, str]]
) -> str:
    for token, table_html in token_table_pairs:
        markdown_text = markdown_text.replace(token, f"\n{table_html}\n")
    return markdown_text


def fix_numbered_headings(markdown_text: str) -> str:
    """
    先頭が「1. **タイトル**」「1.1 **タイトル**」のような行を
    # / ## / ### ... の見出しに変換します。
    """
    lines = markdown_text.splitlines()
    new_lines: List[str] = []

    # 例:
    # "1. **各国の経済状況**" → "# 各国の経済状況"
    # "1.1 **主要国での比較**" → "## 主要国での比較"
    # 太字(** **)が無いケースも一応許容します
    pat = re.compile(r"^\s*(\d+(?:\.\d+)*)\s+(?:\*\*(.+?)\*\*|(.+))\s*$")

    for line in lines:
        m = pat.match(line)
        if m:
            numbering = m.group(1)
            title = m.group(2) or m.group(3) or ""
            level = min(len(numbering.split(".")), 6)
            new_lines.append(f"{'#' * level} {title.strip()}")
        else:
            new_lines.append(line)

    # 先頭に "input" 単独行が残っている場合の対策（安全側）
    if new_lines and new_lines[0].strip().lower() == "input":
        new_lines = new_lines[1:]

    return "\n".join(new_lines)


def main() -> None:
    if not INPUT_HTML.exists():
        raise FileNotFoundError(f"入力HTMLが見つかりません: {INPUT_HTML}")

    soup = load_html(INPUT_HTML)

    # CSSや関連属性を除去 + input等を除去
    remove_css_and_related_attrs(soup)

    # テーブル抽出 & トークン置換
    html_without_tables, token_table_pairs = extract_tables_and_replace_with_tokens(soup)

    # Markdownへ変換
    markdown_text = html_to_markdown(html_without_tables)

    # テーブルHTMLをMarkdownへ差し戻し
    markdown_text = restore_tables_into_markdown(markdown_text, token_table_pairs)

    # 番号付き見出しを # 見出しへ補正
    markdown_text = fix_numbered_headings(markdown_text)

    # 出力
    OUTPUT_MD_DIR.mkdir(parents=True, exist_ok=True)
    with OUTPUT_MD.open("w", encoding="utf-8", newline="\n") as f:
        f.write(markdown_text)

    print(f"変換が完了しました: {OUTPUT_MD}")


if __name__ == "__main__":
    main()
